<a href="https://colab.research.google.com/github/Maraudier/tkfruit/blob/CoLab-Testing/vggClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab library to upload files to notebook
from google.colab import files



In [2]:
!pip install kaggle

In [3]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [4]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
import json
token = {"username":"michaelgrillo","key":"1b1b61b21b7bd669c2b170bf718b9cd8"}
with open ('/content/.kaggle/kaggle.json', 'w') as file:
  json.dump(token, file)

In [0]:
with open ('/content/.kaggle/kaggle.json', 'w') as file:
  json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [9]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [10]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification -p /content

fruits-fresh-and-rotten-for-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
!unzip \*.zip

Archive:  fruits-fresh-and-rotten-for-classification.zip
replace dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 4.59.44 PM.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 4.59.44 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.01.15 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.01.22 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.01.41 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.02.43 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.03.40 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.04.16 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.04.24 PM.png  
  inflating: dataset/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.05.34 PM.png  
  inflating: datase

In [12]:
!ls

dataset  fruits-fresh-and-rotten-for-classification.zip  sample_data


In [0]:
!rm -rf /content/dataset/dataset

In [0]:
import pathlib
train_dir=pathlib.Path('dataset/train')
test_dir=pathlib.Path('dataset/test')

In [15]:
from __future__ import absolute_import, division, print_function, unicode_literals

%tensorflow_version 2.x
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
from tensorflow.keras import datasets, layers, models
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

TensorFlow 2.x selected.


In [16]:
image_count =len(list(train_dir.glob('*/*.png')))+ len(list(test_dir.glob('*/*.png')))
class_names=np.array([item.name for item in train_dir.glob('*') if item.name != 'LICENSE.txt'])
class_names

array(['freshapples', 'freshoranges', 'freshbanana', 'rottenapples',
       'rottenoranges', 'rottenbanana'], dtype='<U13')

In [0]:
test_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator= tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
batch_size= 64
img_height= 224
img_width= 224
steps_per_epoch= np.ceil(image_count/batch_size)

In [18]:
train_data_gen = train_generator.flow_from_directory(directory=str(train_dir),
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     target_size=(img_height, img_width),
                                                     class_mode="sparse",
                                                     classes = list(class_names))
test_data_gen = test_generator.flow_from_directory(directory=str(test_dir),
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     target_size=(img_height, img_width),
                                                     class_mode="sparse",
                                                     classes = list(class_names))

Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


In [19]:
model=models.Sequential()


model.add(layers.Conv2D(input_shape=(224, 224, 3), filters=32, kernel_size=(3,3),
padding="same", activation="relu"))

model.add(layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=32, kernel_size= (3,3), padding="same", activation="relu"))

model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation= "relu"))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation= "relu"))

model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(layers.Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))

model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0

Now we apply the fully connected layer

In [0]:

model.add(layers.Flatten())
model.add(layers.Dense(units=4096, activation="relu"))
model.add(layers.Dense(units=4096, activation="relu"))
model.add(layers.Dense(units=6, activation="softmax"))


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0

In [0]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

history= model.fit_generator(
    train_data_gen,
    epochs=25,
    validation_data=test_data_gen
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 171 steps, validate for 43 steps
Epoch 1/25
171/171 [==============================] - 215s 1s/step - loss: 1.7826 - accuracy: 0.2040 - val_loss: 1.7768 - val_accuracy: 0.2228
Epoch 2/25
171/171 [==============================] - 204s 1s/step - loss: 1.7769 - accuracy: 0.2097 - val_loss: 1.7776 - val_accuracy: 0.1964
Epoch 3/25
171/171 [==============================] - 204s 1s/step - loss: 1.7763 - accuracy: 0.2040 - val_loss: 1.7755 - val_accuracy: 0.2228
Epoch 4/25
171/171 [==============================] - 200s 1s/step - loss: 1.7767 - accuracy: 0.2148 - val_loss: 1.7761 - val_accuracy: 0.2228
Epoch 5/25
171/171 [==============================] - 204s 1s/step - loss: 1.7763 - accuracy: 0.2116 - val_loss: 1.7776 - val_accuracy: 0.2228
Epoch 6/25
171/171 [==============================] - 201s 1s/step - loss: 1.7758 - accuracy: 0.2148 - val_loss: 1.7